In [1]:
import json
import random
import sys
import os

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

random.seed(2)

from src.agents.context_qa_correction_agent import ContextQACorrectionAgent

In [2]:
with open(f"../../../dataset/raw_model_responses/train/train_rajpurkar_squad.json", "r") as f:
    json_data = json.load(f)

In [3]:
json_data[1]

{'task_info': {'type': 'Contextual QA', 'dataset': 'rajpurkar_squad'},
 'additional_info': {'model': 'Meta-Llama-3.1-8B-Instruct',
  'question': 'What is in front of the Notre Dame Main Building?',
  'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'answer': ['a copper statue of Christ'],
  'title': 'University_of_Notre_Dame'},
 'inpu

In [4]:
# index = 190
index = 196
responses = json_data[index]["response"]

question = json_data[index]["additional_info"]["question"]
context = json_data[index]["additional_info"]["context"]
answer = json_data[index]["additional_info"]["answer"]
print(question)
print(answer)

How many individual colleges are part of Notre Dame?
['four']


In [5]:
responses

['There are 5 schools that offer graduate programs and 4 colleges at the undergraduate level, as well as the Architecture School. So, in total, there are 9 individual units: 4 colleges and 5 schools.',
 'There are 4 colleges and 1 Architecture School part of Notre Dame.',
 'There are 4 individual colleges and 1 Architecture School that are part of Notre Dame.',
 'There are 5 individual colleges/schools part of Notre Dame, but the undergraduate component is organized into 4 colleges: Arts and Letters, Science, Engineering, Business, and the Architecture School.',
 'There are 4 individual colleges and 1 Architecture School, plus 5 schools offering graduate programs, and a law school. However, if we are counting the undergraduate colleges, the answer is 4.',
 'There are 5 individual colleges/schools part of Notre Dame: \n\n1. College of Arts and Letters\n2. College of Science\n3. College of Engineering\n4. College of Business\n5. Architecture School',
 'There are 5 individual colleges/sch

In [6]:
agent = ContextQACorrectionAgent(error_detection_only=False)

state = await agent.model.ainvoke({
    "question": question,
    "context": context,
    "answer": answer,
    "responses": responses,
})

2025-08-06 16:17:27.618 | INFO     | src.agents.context_qa_correction_agent:check_for_errors:88 - Errors: [[{'description': 'The question asks about the number of colleges, but the response includes the number of schools as well, which is irrelevant and misleading.', 'location': 'There are 5 schools that offer graduate programs and 4 colleges at the undergraduate level, as well as the Architecture School. So, in total, there are 9 individual units: 4 colleges and 5 schools.', 'correction': 'DELETE: There are 5 schools that offer graduate programs and 4 colleges at the undergraduate level, as well as the Architecture School. So, in total, there are 9 individual units: 4 colleges and 5 schools.'}], [{'description': 'The question asks for the number of colleges, but the response includes the Architecture School, which is not a college.', 'location': '4 colleges and 1 Architecture School', 'correction': 'DELETE: "and 1 Architecture School"'}], [{'description': 'The question asks for the nu

In [7]:
state

{'question': 'How many individual colleges are part of Notre Dame?',
 'context': "Besides its prominence in sports, Notre Dame is also a large, four-year, highly residential research University, and is consistently ranked among the top twenty universities in the United States  and as a major global university. The undergraduate component of the university is organized into four colleges (Arts and Letters, Science, Engineering, Business) and the Architecture School. The latter is known for teaching New Classical Architecture and for awarding the globally renowned annual Driehaus Architecture Prize. Notre Dame's graduate program has more than 50 master's, doctoral and professional degree programs offered by the five schools, with the addition of the Notre Dame Law School and a MD-PhD program offered in combination with IU medical School. It maintains a system of libraries, cultural venues, artistic and scientific museums, including Hesburgh Library and the Snite Museum of Art. Over 80% o